In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from scipy import stats
data_dir = "/scratch/new_courier_pilot/"
subject='R1620J'
exp = 'EFRCourierOpenLoop'
session= 3
reader = cml.CMLReader(subject, exp,3)
evs = reader.load('task_events')
exp = 'EFRCourierOpenLoop'

# Get unique subjects for the experiment
subjects = whole_df.query('experiment == @exp')['subject'].unique()
subjects=['R1620J', 'R1653J', 'R1693J',
       'R1709J', 'R1724J']
# Initialize results storage
results = []
results_2=[]
# Loop through each subject
for subject in subjects:
  #  print(f"Processing subject: {subject}")
    
    # Get subject data
    sub_df = whole_df.query('experiment == @exp and subject == @subject')
  
    # Get all sessions for this subject
    sessions = sub_df['session'].unique()
    
    # Loop through each session
    for session in sessions:
        reader = cml.CMLReader(subject, exp, session=session)
        evs = reader.load('task_events')
        evs['inside_stimuli'] = -999
        stimuli_indices = evs[evs['type'] == 'STIM'].index

        for i in stimuli_indices:
            current_offset = evs.loc[i, 'eegoffset']
            evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
            # Keep checking subsequent rows until difference >= 3000
            j = i+1
            rows_to_assign = []  # Store indices of rows to assign

            while j < len(evs):
                next_offset = evs.loc[j, 'eegoffset']
                time_diff = abs(next_offset - current_offset)
                #print(time_diff)
                if time_diff < 3000:
                    # Still within 3 seconds, add this row to assignment list
                    rows_to_assign.append(j)
                    j += 1
                else:
                    # Found a row with difference >= 4000, stop checking
                    break

        # Assign burst_freq to all rows that were within 4000ms of the STIM
        for row_idx in rows_to_assign:
            evs.loc[row_idx, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
            evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
        # Get unique trials for this session
            trials = evs['trial'].unique()
            # Find indices of all WORD events
            word_indices = evs[evs['type'] == "WORD"].index

            # Get the events immediately before WORD events
            preceding_events = evs.iloc[word_indices - 1]

            # Find which preceding events are STIM type
            stim_mask = preceding_events['type'] == 'STIM'
            stim_indices = preceding_events[stim_mask].index

            # Update the inside_stimuli column for WORD events that follow STIM events
            for stim_idx in stim_indices:
                word_idx = stim_idx + 1
                burst_freq = evs.loc[stim_idx, 'stim_params']['burst_freq']
                evs.loc[word_idx, 'inside_stimuli'] = burst_freq
                evs.loc[stim_idx, 'inside_stimuli'] = burst_freq
            FirstTrial=evs[evs['trial']==1]
            pd.set_option('display.max_rows', None)
            FirstTrial[['store','type']]
            Events=evs[evs['type']=='pointing finished']
            Events=Events.reset_index(drop=True)
            Words=evs[evs['type']=='WORD']
            Words=Words.reset_index(drop=True)
            Words['new_time']=Events['mstime']
            Words['inside_stimuli']
        # Select the columns (assuming Words is a DataFrame)


    # Get unique stores (add parentheses to call the method)
            allstore = Words['store'].unique()

    # Initialize empty list to collect results
            result_list = []

            for store in allstore:
                # Filter data for current store (add missing closing bracket)
                new_words = Words[Words['store'] == store]

                # Loop through rows (use range(len()) for iteration)
                for i in range(len(new_words)):
                    result = {
                        'store': new_words.iloc[i]['store'],
                        # Fix time calculation (this gives 0 - did you mean something else?)
                        'time': new_words.iloc[i]['mstime'] - new_words.iloc[i]['new_time'],  
                        # Fix column reference (assuming you meant a column from new_words)
                        'condition': new_words.iloc[i]['inside_stimuli'],  # or whatever column you need
                        'visit_time': i,
                        'trial':new_words.iloc[i]['trial'],
                        'session':new_words.iloc[i]['session'],
                        'session':new_words.iloc[i]['subject']
                    }
                    result_list.append(result)

# Convert to DataFrame at the end (more efficient than repeated append)
result_pd = pd.DataFrame(result_list)
        

In [ ]:
result_pd

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from scipy import stats

data_dir = "/scratch/new_courier_pilot/"
subject = 'R1620J'
exp = 'EFRCourierOpenLoop'
session = 3
reader = cml.CMLReader(subject, exp, 3)
evs = reader.load('task_events')
exp = 'EFRCourierOpenLoop'
whole_df = cml.CMLReader.get_data_index()
# Get unique subjects for the experiment
result_list = pd.DataFrame()
subjects = ['R1620J', 'R1653J', 'R1693J', 'R1709J', 'R1724J']
allstore=[]
# Initialize results storage
results = []
results_2 = []
Words_all = pd.DataFrame()
# Loop through each subject
for subject in subjects:
    # print(f"Processing subject: {subject}")
    
    # Get subject data
    sub_df = whole_df.query('experiment == @exp and subject == @subject')
    
    # Get all sessions for this subject
    sessions = sub_df['session'].unique()
    
    # Loop through each session
    for session in sessions:
        reader = cml.CMLReader(subject, exp, session=session)
        evs = reader.load('task_events')
        evs['inside_stimuli'] = -999
        stimuli_indices = evs[evs['type'] == 'STIM'].index

        for i in stimuli_indices:
            current_offset = evs.loc[i, 'eegoffset']
            evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
            
            # Keep checking subsequent rows until difference >= 3000
            j = i + 1
            rows_to_assign = []  # Store indices of rows to assign

            while j < len(evs):
                next_offset = evs.loc[j, 'eegoffset']
                time_diff = abs(next_offset - current_offset)
                # print(time_diff)
                if time_diff < 3000:
                    # Still within 3 seconds, add this row to assignment list
                    rows_to_assign.append(j)
                    j += 1
                else:
                    # Found a row with difference >= 4000, stop checking
                    break

            # Assign burst_freq to all rows that were within 4000ms of the STIM
            for row_idx in rows_to_assign:
                evs.loc[row_idx, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
                evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']

        # Get unique trials for this session
        trials = evs['trial'].unique()
        
        # Find indices of all WORD events
        word_indices = evs[evs['type'] == "WORD"].index

        # Get the events immediately before WORD events
        preceding_events = evs.iloc[word_indices - 1]

        # Find which preceding events are STIM type
        stim_mask = preceding_events['type'] == 'STIM'
        stim_indices = preceding_events[stim_mask].index

        # Update the inside_stimuli column for WORD events that follow STIM events
        for stim_idx in stim_indices:
            word_idx = stim_idx + 1
            burst_freq = evs.loc[stim_idx, 'stim_params']['burst_freq']
            evs.loc[word_idx, 'inside_stimuli'] = burst_freq
            evs.loc[stim_idx, 'inside_stimuli'] = burst_freq
            
        FirstTrial = evs[evs['trial'] == 1]
        pd.set_option('display.max_rows', None)
        FirstTrial[['store', 'type']]
        trials=evs['trial'].unique()
        trials = [x for x in trials if x != -999]
        for trial in trials:
            evs_trial=evs[evs['trial'] == trial]
            pointingfinish = evs_trial[evs_trial['type'] == 'pointing finished']
            pointingfinish = pointingfinish.reset_index(drop=True)
            pointingfinish = pointingfinish.iloc[0:-1].reset_index(drop=True)
            Words = evs_trial[evs_trial['type'] == 'WORD']
            Words = Words.reset_index(drop=True)
            pointingfinish['arrival'] = Words['mstime']
            pointingfinish['store']=Words['store']
            pointingfinish['condition']=Words['inside_stimuli']
            Words_all = Words_all.append(pointingfinish)
            # Get unique stores (add parentheses to call the method)
        allstore = Words_all['store'].unique()

    # Initialize empty list to collect results


        for store in allstore:
            # Filter data for current store
            new_words = Words_all[Words_all['store'] == store]

            # Loop through rows
            for i in range(len(new_words)):
                result = {
                    'store': new_words.iloc[i]['store'],
                    # Fix time calculation
                    'time': new_words.iloc[i]['arrival'] - new_words.iloc[i]['mstime'],  
                    # Fix column reference
                    'condition': new_words.iloc[i]['condition'],
                    'visit_time': i,
                    'trial': new_words.iloc[i]['trial'],
                    'session': new_words.iloc[i]['session'],
                    'subject': new_words.iloc[i]['subject']  # Fixed: was overwriting 'session'
                }
                result_list = result_list.append(result, ignore_index=True)

        # Convert to DataFrame at the end (more efficient than repeated append)
    result_pd = pd.DataFrame(result_list)

In [51]:
result_pd[(result_pd['subject']=='R1620J')&( result_pd['session']==3)&( result_pd['trial']==0)]

,store,time,condition,visit_time,trial,session,subject
0,pharmacy,20471.0,3.0,0.0,0.0,3.0,R1620J
5,dentist,11610.0,-999.0,0.0,0.0,3.0,R1620J
10,music_store,17041.0,-999.0,0.0,0.0,3.0,R1620J
15,jewelry_store,15224.0,-999.0,0.0,0.0,3.0,R1620J
19,grocery_store,13600.0,3.0,0.0,0.0,3.0,R1620J
23,florist,16740.0,-999.0,0.0,0.0,3.0,R1620J
26,cafe,14012.0,-999.0,0.0,0.0,3.0,R1620J
29,pizzeria,20443.0,-999.0,0.0,0.0,3.0,R1620J
33,gym,10000.0,3.0,0.0,0.0,3.0,R1620J
37,bike_shop,16200.0,-999.0,0.0,0.0,3.0,R1620J


In [ ]:
subject='R1620J

reader = cml.CMLReader(subject, exp, session=session)
evs_2 = reader.load('task_events')
trial2=evs_2[evs_2['trial'] == 2]
print(len(trial2[trial2['type']=='WORD']))
print(len(trial2[trial2['type']=='pointing finished']))

In [59]:
reader = cml.CMLReader('R1620J', exp, session=3)
evs_2 = reader.load('task_events')
evs_3=evs_2[evs_2['trial']==0]
evs_3[['type','mstime','store']]
1654633396181-1654633375710


20471

In [ ]:
pointingfinish = evs_trial[evs_trial['type'] == 'pointing finished']
pointingfinish

In [ ]:
trial

In [ ]:
# Select the columns (assuming Words is a DataFrame)


# Get unique stores (add parentheses to call the method)
allstore = Words['store'].unique()

# Initialize empty list to collect results
result_list = []

for store in allstore:
    # Filter data for current store (add missing closing bracket)
    new_words = Words[Words['store'] == store]
    
    # Loop through rows (use range(len()) for iteration)
    for i in range(len(new_words)):
        result = {
            'store': new_words.iloc[i]['store'],
            # Fix time calculation (this gives 0 - did you mean something else?)
            'time': new_words.iloc[i]['mstime'] - new_words.iloc[i]['new_time'],  
            # Fix column reference (assuming you meant a column from new_words)
            'condition': new_words.iloc[i]['inside_stimuli'],  # or whatever column you need
            'condition_Stim': 0 if new_words.iloc[i]['inside_stimuli']==-999 else 1,
            'visit_time': i,
            'trial':new_words.iloc[i]['trial'],
            'session':new_words.iloc[i]['session'],
            'session':new_words.iloc[i]['subject']
        }
        result_list.append(result)

# Convert to DataFrame at the end (more efficient than repeated append)
result_pd = pd.DataFrame(result_list)
result_pd    

In [ ]:
new_words.iloc[i]['mstime']

In [ ]:
new = result_pd.groupby(["condition", "visit_time"]).mean()
new = result_pd.groupby(["condition", "visit_time"]).mean()
new_se = result_pd.groupby(["condition", "visit_time"]).sem()
new

In [ ]:
plt.figure(figsize=(10, 6))

# Get unique condition_Stim values
condition_stim_values = new.index.get_level_values('condition_Stim').unique()

# Plot each condition_Stim as a different line
for condition in condition_stim_values:
    # Filter data for this condition
    condition_data = new.loc[condition]
    
    # Get visit_time values (x-axis) and the values to plot (y-axis)
    visit_times = condition_data.index  # visit_time values
    
    # Choose which column to plot - adjust this based on your data
    # For example, if you want to plot a specific column:
    y_values = condition_data.iloc[:, 0]  # plots first column, or specify column name
    
    # Plot the line
    plt.plot(visit_times, y_values, marker='o', label=f'Condition Stim {condition}', linewidth=2, markersize=6)

# Customize the plot
plt.xlabel('Visit Time', fontsize=12)
plt.ylabel('Mean Time', fontsize=12)
plt.title('Mean Time by Condition Stim and Visit Time', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(int(new.index.get_level_values('visit_time').min()), 
                 int(new.index.get_level_values('visit_time').max()) + 1))

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Calculate both mean and standard error
new = result_pd.groupby(["condition", "visit_time"]).mean()
new_se = result_pd.groupby(["condition", "visit_time"]).sem()

# Create the plot
plt.figure(figsize=(10, 6))

# Get unique condition_Stim values
condition_stim_values = new.index.get_level_values('condition').unique()

# Plot each condition_Stim as a different line with error bars
for condition in condition_stim_values:
    # Filter data for this condition
    condition_data = new.loc[condition]
    condition_se_data = new_se.loc[condition]
    
    # Get visit_time values (x-axis) and the values to plot (y-axis)
    visit_times = condition_data.index  # visit_time values
    
    # Plot only the 'time' column
    y_values = condition_data['time']
    error_values = condition_se_data['time']
    
    # Plot the line with error bars
    plt.errorbar(visit_times, y_values, yerr=error_values, marker='o', 
                label=f'Condition Stim {condition}', linewidth=2, markersize=6,
                capsize=5, capthick=1.5)

# Customize the plot
plt.xlabel('Visit Time', fontsize=12)
plt.ylabel('Mean Time', fontsize=12)
plt.title('Mean Time by Condition Stim and Visit Time', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(int(new.index.get_level_values('visit_time').min()), 
                 int(new.index.get_level_values('visit_time').max()) + 1))

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Get unique condition_Stim values
condition_stim_values = new.index.get_level_values('condition_Stim').unique()

# Plot each condition_Stim as a different line
for condition in condition_stim_values:
    # Filter data for this condition
    condition_data = new.loc[condition]
    
    # Get visit_time values (x-axis) and the values to plot (y-axis)
    visit_times = condition_data.index  # visit_time values
    
    # Choose which column to plot - adjust this based on your data
    # For example, if you want to plot a specific column:
    y_values = condition_data.iloc[:, 0]  # plots first column, or specify column name
    
    # Plot the line
    plt.plot(visit_times, y_values, marker='o', label=f'Condition Stim {condition}', linewidth=2, markersize=6)

# Customize the plot
plt.xlabel('Visit Time', fontsize=12)
plt.ylabel('Mean Time', fontsize=12)
plt.title('Mean Time by Condition Stim and Visit Time', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(int(new.index.get_level_values('visit_time').min()), 
                 int(new.index.get_level_values('visit_time').max()) + 1))

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
FirstTrial = evs[evs['trial'] == 1]
pd.set_option('display.max_rows', None)
FirstTrial[['store', 'type']]

Events = evs[evs['type'] == 'pointing finished']
Words = evs[evs['type'] == 'WORDS']

# Fix the alignment issue - need to match indices or use merge/join
# This will only work if Events and Words have the same length and order
# Words['new_time'] = Events['mstime'].values  # Use .values to avoid index alignment

# Better approach - use merge or explicit index matching:
Words_new = Words.copy()  # Avoid SettingWithCopyWarning
Words_new['new_time'] = Events['mstime'].reset_index(drop=True)

In [ ]:
Words_new

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Assuming you have your 'new' DataFrame from the groupby operation
# new = result_pd.groupby(["condition", "visit_time"]).mean()

# If your data needs to be reset from MultiIndex:
# new_reset = new.reset_index()

def plot_interaction(data, metric='time', figsize=(12, 8)):
    """
    Plot the interaction between stimulation condition and visit time
    
    Parameters:
    data: DataFrame with columns ['condition', 'visit_time', metric]
    metric: string, the column to plot ('time', 'trial', 'session')
    """
    
    # Create the figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Get unique conditions and sort them
    conditions = sorted(data['condition'].unique())
    
    # Define colors for each condition
    colors = plt.cm.Set1(np.linspace(0, 1, len(conditions)))
    condition_colors = dict(zip(conditions, colors))
    
    # Plot lines for each condition
    for i, condition in enumerate(conditions):
        condition_data = data[data['condition'] == condition].sort_values('visit_time')
        
        # Handle condition labels
        if condition == -999:
            label = 'No Stimulation'
            linestyle = '--'
            linewidth = 3
        else:
            label = f'{condition}Hz'
            linestyle = '-'
            linewidth = 2.5
        
        # Plot the line
        ax.plot(condition_data['visit_time'], 
                condition_data[metric],
                marker='o', 
                markersize=8,
                linewidth=linewidth,
                linestyle=linestyle,
                color=condition_colors[condition],
                label=label,
                alpha=0.8)
    
    # Customize the plot
    ax.set_xlabel('Visit Time (Sequential Order)', fontsize=14, fontweight='bold')
    
    # Set y-axis label based on metric
    ylabel_map = {
        'time': 'Response Time (ms)',
        'trial': 'Trial Number', 
        'session': 'Session Number'
    }
    ax.set_ylabel(ylabel_map.get(metric, metric), fontsize=14, fontweight='bold')
    
    # Set title
    ax.set_title(f'Stimulation Condition × Visit Time Interaction\n({ylabel_map.get(metric, metric)})', 
                fontsize=16, fontweight='bold', pad=20)
    
    # Add grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    
    # Customize legend
    ax.legend(title='Stimulation Condition', 
              title_fontsize=12,
              fontsize=11,
              loc='best',
              frameon=True,
              fancybox=True,
              shadow=True)
    
    # Handle negative visit times on x-axis
    x_ticks = sorted(data['visit_time'].unique())
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([str(int(x)) for x in x_ticks])
    
    # Add vertical line at visit_time = 0 if it exists in the data
    if 0 in x_ticks:
        ax.axvline(x=0, color='red', linestyle=':', alpha=0.7, linewidth=2)
        ax.text(0.1, ax.get_ylim()[1]*0.95, 'Baseline', rotation=90, 
                verticalalignment='top', fontsize=10, color='red')
    
    # Highlight significant interaction regions (you can customize this)
    # This is a simplified approach - you might want to add actual statistical testing
    highlight_regions = []
    visit_times = sorted(data['visit_time'].unique())
    
    for i in range(len(visit_times)-1):
        start_time = visit_times[i]
        end_time = visit_times[i+1]
        
        # Get values for each condition at these time points
        start_vals = []
        end_vals = []
        
        for condition in conditions:
            start_val = data[(data['condition'] == condition) & 
                           (data['visit_time'] == start_time)][metric]
            end_val = data[(data['condition'] == condition) & 
                         (data['visit_time'] == end_time)][metric]
            
            if len(start_val) > 0 and len(end_val) > 0:
                start_vals.append(start_val.iloc[0])
                end_vals.append(end_val.iloc[0])
        
        # Calculate slopes for each condition
        if len(start_vals) >= 2:
            slopes = [(end_vals[j] - start_vals[j]) for j in range(len(start_vals))]
            slope_var = np.var(slopes)
            
            # If slope variance is high, highlight this region
            if slope_var > np.std([data[metric].std()]):  # Adjust threshold as needed
                ax.axvspan(start_time, end_time, alpha=0.2, color='yellow')
                ax.text((start_time + end_time)/2, ax.get_ylim()[1]*0.9, 
                       'Significant\nInteraction', 
                       horizontalalignment='center',
                       fontsize=9, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))
    
    plt.tight_layout()
    return fig, ax

def plot_heatmap_interaction(data, metric='time', figsize=(10, 6)):
    """
    Create a heatmap showing the interaction pattern
    """
    # Pivot the data for heatmap
    heatmap_data = data.pivot(index='condition', columns='visit_time', values=metric)
    
    # Create the heatmap
    fig, ax = plt.subplots(figsize=figsize)
    
    # Custom colormap
    cmap = sns.diverging_palette(250, 10, as_cmap=True)
    
    sns.heatmap(heatmap_data, 
                annot=True, 
                fmt='.1f', 
                cmap=cmap,
                center=heatmap_data.mean().mean(),
                ax=ax,
                cbar_kws={'label': f'{metric}'})
    
    # Customize labels
    ax.set_xlabel('Visit Time', fontsize=12, fontweight='bold')
    ax.set_ylabel('Stimulation Condition', fontsize=12, fontweight='bold')
    ax.set_title(f'Interaction Heatmap: {metric} by Condition and Visit Time', 
                fontsize=14, fontweight='bold')
    
    # Handle condition labels
    y_labels = []
    for condition in heatmap_data.index:
        if condition == -999:
            y_labels.append('No Stim')
        else:
            y_labels.append(f'{condition}Hz')
    ax.set_yticklabels(y_labels, rotation=0)
    
    plt.tight_layout()
    return fig, ax

# Example usage with your data:
# Assuming your grouped data is in 'new' DataFrame

# Method 1: If 'new' is a MultiIndex DataFrame from groupby
# new_reset = new.reset_index()

# Method 2: If you need to create sample data to test (remove this when using real data)
def create_sample_data():
    """Create sample data for testing - replace this with your actual data"""
    conditions = [-999, 25, 50, 100, 130]
    visit_times = list(range(-3, 4))  # Negative to positive visit times
    
    data = []
    for condition in conditions:
        for visit_time in visit_times:
            # Simulate interaction effect
            if condition == -999:
                time_val = 2000 + (visit_time * 100)  # Linear increase
            elif condition <= 50:
                time_val = 2200 - (visit_time * 80)   # Decreasing with practice
            else:
                time_val = 1900 + (visit_time**2 * 20)  # U-shaped
            
            data.append({
                'condition': condition,
                'visit_time': visit_time,
                'time': time_val + np.random.normal(0, 50),
                'trial': 15 + np.random.normal(0, 2),
                'session': 2.5 + np.random.normal(0, 0.5)
            })
    
    return pd.DataFrame(data)

# Create or load your data
# Replace this line with: plot_data = new.reset_index()
plot_data = create_sample_data()  # Remove this line when using real data

# Create the plots
print("Creating interaction line plot...")
fig1, ax1 = plot_interaction(plot_data, metric='time', figsize=(14, 8))
plt.show()

print("\nCreating interaction heatmap...")
fig2, ax2 = plot_heatmap_interaction(plot_data, metric='time', figsize=(12, 6))
plt.show()

# Statistical analysis of the interaction
print("\n" + "="*50)
print("STATISTICAL ANALYSIS")
print("="*50)

# Perform ANOVA to test interaction significance
from scipy.stats import f_oneway
from itertools import combinations

# Test main effects and interaction (simplified approach)
# For proper mixed-effects analysis, use statsmodels
conditions = plot_data['condition'].unique()
visit_times = plot_data['visit_time'].unique()

print(f"Conditions tested: {sorted(conditions)}")
print(f"Visit times: {sorted(visit_times)}")
print(f"Sample size per condition×visit_time: {len(plot_data) // (len(conditions) * len(visit_times))}")

# Test if slopes differ between conditions (simplified interaction test)
slopes = {}
for condition in con